# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aguimes,27.91,-15.45,69.01,31,0,9.17,ES,1579405367
1,1,meulaboh,4.14,96.13,81.50,79,9,6.51,ID,1579405367
2,2,punta arenas,-53.15,-70.92,51.80,62,75,18.34,CL,1579405367
3,3,paramonga,-10.67,-77.83,69.08,90,81,7.02,PE,1579405367
4,4,broome,42.25,-75.83,33.01,79,90,10.29,US,1579405368


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
#configure gmpas
gmaps.configure(api_key = g_key)

In [24]:
#heat map of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity=300, point_radius =5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
#narrow down cities that fit criteria and drop any results with no value 
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"]<80) & (city_data_df["Max Temp"] > 70) \
                                & (city_data_df["Wind Speed"]<10) \
                                & (city_data_df["Cloudiness"] == 0)].dropna()

narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,tabou,4.42,-7.35,76.24,93,0,1.21,CI,1579405370
82,82,mumford,5.26,-0.76,76.84,89,0,2.08,GH,1579405386
90,90,sao luiz gonzaga,-28.41,-54.96,73.92,72,0,4.94,BR,1579405388
119,119,cantaura,9.31,-64.36,75.63,71,0,6.89,VE,1579405394
185,185,edd,13.93,41.69,72.77,66,0,4.63,ER,1579405410
306,306,axim,4.87,-2.24,76.82,91,0,2.15,GH,1579405439
351,351,kapoeta,4.77,33.59,77.13,23,0,4.94,SS,1579405449
432,432,sotouboua,8.57,0.98,70.32,27,0,3.91,TG,1579405469


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
#create a data frame hotel_df to store hotel names w city
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] =""
hotel_df

,City,Country,Lat,Lng,Hotel Name
14,tabou,CI,4.42,-7.35,
82,mumford,GH,5.26,-0.76,
90,sao luiz gonzaga,BR,-28.41,-54.96,
119,cantaura,VE,9.31,-64.36,
185,edd,ER,13.93,41.69,
306,axim,GH,4.87,-2.24,
351,kapoeta,SS,4.77,33.59,
432,sotouboua,TG,8.57,0.98,


In [27]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f'{lat}, {lng}'
    
    # use the search term: hotel and out lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    name_address = requests .get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except(KeyError, IndexError):
        print("Missing field/result...skipping")
        
hotel_df

Missing field/result...skipping


,City,Country,Lat,Lng,Hotel Name
14,tabou,CI,4.42,-7.35,hotêl doufoulougou
82,mumford,GH,5.26,-0.76,Spanish Villa
90,sao luiz gonzaga,BR,-28.41,-54.96,Xerife Hotel
119,cantaura,VE,9.31,-64.36,Hotel Lorenzana
185,edd,ER,13.93,41.69,
306,axim,GH,4.87,-2.24,AXIM BEACH RESORT AND SPA
351,kapoeta,SS,4.77,33.59,Eastgate Hotel
432,sotouboua,TG,8.57,0.98,Hôtel La Planète


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))